In [1]:
import pandas as pd

df = pd.read_csv('data/rated_2014-01_initial_cleaning.csv')
df.info()


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/usr/lib/python3.11/runpy.py", line 198, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.11/runpy.py", line 88, in _run_code
    exec(code, run_globals)
  File "/home/zamog/.cache/pypoetry/virtualenvs/non-package-mode-uH4eZcH1-py3.11/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/zamog/.cache/pypoetry/virtualenvs/non-package-mode-uH4eZcH1-py3.11/lib/python3.11/site-packages/traitlets/config/applicat

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/usr/lib/python3.11/runpy.py", line 198, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.11/runpy.py", line 88, in _run_code
    exec(code, run_globals)
  File "/home/zamog/.cache/pypoetry/virtualenvs/non-package-mode-uH4eZcH1-py3.11/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/zamog/.cache/pypoetry/virtualenvs/non-package-mode-uH4eZcH1-py3.11/lib/python3.11/site-packages/traitlets/config/applicat

AttributeError: _ARRAY_API not found

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 697470 entries, 0 to 697469
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   event         697470 non-null  object 
 1   white_elo     697470 non-null  float64
 2   black_elo     697470 non-null  float64
 3   time_control  697470 non-null  object 
 4   result        697470 non-null  object 
 5   termination   697470 non-null  object 
 6   moves         697470 non-null  object 
dtypes: float64(2), object(5)
memory usage: 37.2+ MB


### Choosing only classical games and deleting `event` column

In [2]:
df_classical = df[df['event'] == 'classical']
df_classical = df_classical[df_classical.columns[1:]]
df_classical.info()

<class 'pandas.core.frame.DataFrame'>
Index: 246495 entries, 2 to 697466
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   white_elo     246495 non-null  float64
 1   black_elo     246495 non-null  float64
 2   time_control  246495 non-null  object 
 3   result        246495 non-null  object 
 4   termination   246495 non-null  object 
 5   moves         246495 non-null  object 
dtypes: float64(2), object(4)
memory usage: 13.2+ MB


### Dividing time_control into two columns

In [3]:
df_classical['time_control'].head()

2     900+30
7      600+0
8      600+0
12    1800+8
13    1200+5
Name: time_control, dtype: object

In [4]:
df_classical[['time', 'increment']] = df_classical['time_control'].str.split('+', expand=True)

df_classical['time'] = pd.to_numeric(df_classical['time'])
df_classical['increment'] = pd.to_numeric(df_classical['increment'])

df_classical = df_classical.drop(columns=['time_control'])
df_classical.info()

<class 'pandas.core.frame.DataFrame'>
Index: 246495 entries, 2 to 697466
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   white_elo    246495 non-null  float64
 1   black_elo    246495 non-null  float64
 2   result       246495 non-null  object 
 3   termination  246495 non-null  object 
 4   moves        246495 non-null  object 
 5   time         246495 non-null  int64  
 6   increment    246495 non-null  int64  
dtypes: float64(2), int64(2), object(3)
memory usage: 15.0+ MB


### Normalizing time and elo values.

In [5]:
from sklearn.preprocessing import MinMaxScaler
import joblib

time_cols = ['time', 'increment']
elo_cols = ['white_elo', 'black_elo']

scaler = MinMaxScaler()
df_classical[time_cols] = scaler.fit_transform(df_classical[time_cols])

joblib.dump(scaler, 'models/time_control_scaler.save')

df_classical[elo_cols] = scaler.fit_transform(df_classical[elo_cols])
joblib.dump(scaler, 'models/elo_scaler.save')

['models/elo_scaler.save']

In [6]:
df_classical.head()

,white_elo,black_elo,result,termination,moves,time,increment
2,0.477483,0.280130,1-0,normal,e4 d5 e5 Nc6 d4 g6 Bg5 Bg7 Nd2 f6 exf6 Nxf6 Nb...,0.083333,0.166667
7,0.394818,0.581759,1-0,normal,e4 e6 Nc3 c6 f4 d5 f5 exf5 exd5 cxd5 d4 Qe7+ B...,0.055556,0.000000
8,0.455275,0.594788,1-0,normal,e4 g6 Bc4 Bg7 c3 Nf6 e5 Ng4 d4 d5 Bb3 Nc6 h3 N...,0.055556,0.000000
12,0.335595,0.551792,0-1,normal,d4 Nf6 c4 a6 Nc3 b6 e4 Bb7 e5 Ng8 Nh3 h6 Be2 e...,0.166667,0.044444
13,0.453424,0.164169,1/2-1/2,normal,d4 d5 f4 Nc6 Nc3 Nxd4 Nxd5 Qxd5,0.111111,0.027778


### Creating one-hot encoding for game results and termination.

In [7]:
df_classical = pd.get_dummies(df_classical, columns=['result', 'termination'])
one_hot_cols = ['result_0-1','result_1-0','result_1/2-1/2','termination_normal','termination_rules infraction','termination_time forfeit']
df_classical[one_hot_cols] = df_classical[one_hot_cols].astype('float32') 
df_classical.head()

,white_elo,black_elo,moves,time,increment,result_0-1,result_1-0,result_1/2-1/2,termination_normal,termination_rules infraction,termination_time forfeit
2,0.477483,0.280130,e4 d5 e5 Nc6 d4 g6 Bg5 Bg7 Nd2 f6 exf6 Nxf6 Nb...,0.083333,0.166667,0.0,1.0,0.0,1.0,0.0,0.0
7,0.394818,0.581759,e4 e6 Nc3 c6 f4 d5 f5 exf5 exd5 cxd5 d4 Qe7+ B...,0.055556,0.000000,0.0,1.0,0.0,1.0,0.0,0.0
8,0.455275,0.594788,e4 g6 Bc4 Bg7 c3 Nf6 e5 Ng4 d4 d5 Bb3 Nc6 h3 N...,0.055556,0.000000,0.0,1.0,0.0,1.0,0.0,0.0
12,0.335595,0.551792,d4 Nf6 c4 a6 Nc3 b6 e4 Bb7 e5 Ng8 Nh3 h6 Be2 e...,0.166667,0.044444,1.0,0.0,0.0,1.0,0.0,0.0
13,0.453424,0.164169,d4 d5 f4 Nc6 Nc3 Nxd4 Nxd5 Qxd5,0.111111,0.027778,0.0,0.0,1.0,1.0,0.0,0.0


### Creating move embeddings with fastText model.

In [8]:
EMBEDDINGS_MODEL = 'models/chess2vec-3ws-1dim.bin'
import fasttext as ft

model = ft.load_model(EMBEDDINGS_MODEL)

In [9]:
from pandas import DataFrame
import numpy as np
def process_game_moves(vecs_dictionary: dict, moves_str: str):
    moves = moves_str.split(' ')

    return np.array([vecs_dictionary[move] for move in moves], dtype='float32')

def moves2vec(model, df_moves: DataFrame):
    unique_moves = set()
    for moves in df_moves.values:
        unique_moves.update(moves.split())
    
    moves_vecs = {move: model.get_word_vector(move) for move in unique_moves}
    
    return df_moves.apply(lambda moves_str: process_game_moves(moves_vecs, moves_str).flatten())

In [10]:
df_classical['moves'] = moves2vec(model, df_classical['moves'])
df_classical.head()

,white_elo,black_elo,moves,time,increment,result_0-1,result_1-0,result_1/2-1/2,termination_normal,termination_rules infraction,termination_time forfeit
2,0.477483,0.280130,"[-0.95061314, -0.9539115, -0.9600917, -0.95966...",0.083333,0.166667,0.0,1.0,0.0,1.0,0.0,0.0
7,0.394818,0.581759,"[-0.95061314, -0.94958806, -0.9628587, -0.9650...",0.055556,0.000000,0.0,1.0,0.0,1.0,0.0,0.0
8,0.455275,0.594788,"[-0.95061314, -0.9661038, -0.96249336, -0.9604...",0.055556,0.000000,0.0,1.0,0.0,1.0,0.0,0.0
12,0.335595,0.551792,"[-0.9569214, -0.95958155, -0.9672969, -0.97700...",0.166667,0.044444,1.0,0.0,0.0,1.0,0.0,0.0
13,0.453424,0.164169,"[-0.9569214, -0.9539115, -0.97433335, -0.95966...",0.111111,0.027778,0.0,0.0,1.0,1.0,0.0,0.0


In [34]:
df_moves = df_classical['moves']

In [11]:
df_classical.to_hdf('data/classical_1dim.h5', key='df', mode='w', index=False)

/tmp/ipykernel_164041/1920346002.py:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['moves'], dtype='object')]

  df_classical.to_hdf('data/classical_1dim.h5', key='df', mode='w', index=False)


In [50]:
len(df_classical['moves'].values[0])

45